In [1]:
from electroviz.core.experiment import Experiment

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("depth")

In [ ]:
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import PairSummary
import matplotlib.pyplot as plt

stims = [stimuli[2], stimuli[1], stimuli[4], stimuli[0]]

for ctx_idx, ctx_unit in enumerate(pop0_sort[100:200]):

    ctx_kernels = [SparseNoiseKernel(ctx_unit, stims[0]), 
                   SparseNoiseKernel(ctx_unit, stims[1]), 
                   StaticGratingsKernel(ctx_unit, stims[2]), 
                   StaticGratingsKernel(ctx_unit, stims[3])]

    for lgn_idx, lgn_unit in enumerate(pop0_sort[600:700]):

        lgn_kernels = [SparseNoiseKernel(lgn_unit, stims[0]), 
                       SparseNoiseKernel(lgn_unit, stims[1]), 
                       StaticGratingsKernel(lgn_unit, stims[2]), 
                       StaticGratingsKernel(lgn_unit, stims[3])]

        file_path = "G:/Shared drives/Neuropixels/Figures/RG001/PairSummary/"
        file_name = "PairSummary_{0}_{1}.png".format(str(ctx_idx + 100), str(lgn_idx + 600))

        PairSummary(stims, (ctx_unit, lgn_unit), (ctx_kernels, lgn_kernels), save_path=file_path+file_name)

        plt.close("all")

In [42]:
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import PairSummary

stims = [stimuli[2], stimuli[1], stimuli[4], stimuli[0]]

ctx_unit = pop0_sort[130]
ctx_kernels = [SparseNoiseKernel(ctx_unit, stims[0]), 
                SparseNoiseKernel(ctx_unit, stims[1]), 
                StaticGratingsKernel(ctx_unit, stims[2]), 
                StaticGratingsKernel(ctx_unit, stims[3])]

lgn_unit = pop0_sort[685]
lgn_kernels = [SparseNoiseKernel(lgn_unit, stims[0]), 
                SparseNoiseKernel(lgn_unit, stims[1]), 
                StaticGratingsKernel(lgn_unit, stims[2]), 
                StaticGratingsKernel(lgn_unit, stims[3])]

PairSummary(stims, (ctx_unit, lgn_unit), (ctx_kernels, lgn_kernels))

In [ ]:
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.utils.cross_corr import cross_corr
import pandas as pd

xcorr_df = pd.DataFrame()

for ctx_idx, ctx_unit in enumerate(pop0_sort[50:150]):
    for lgn_idx, lgn_unit in enumerate(pop0_sort[625:725]):
        xcorr_raw, xcorr_filt = cross_corr((ctx_unit, lgn_unit))
        peak_raw, base_raw = xcorr_raw[52:56].mean(), xcorr_raw[47:51].mean()
        peak_filt, base_filt_mean, base_filt_max = xcorr_filt[52:56].max(), xcorr_filt[47:51].mean(), xcorr_filt[47:51].max()
        df_add = pd.DataFrame({"ctx" : [ctx_idx + 50], 
                               "lgn" : [lgn_idx + 625], 
                               "raw" : [peak_raw / base_raw], 
                               "filt_mean_full": [peak_filt - xcorr_filt.mean()], 
                               "filt_mean": [peak_filt - base_filt_mean], 
                               "filt_max": [peak_filt - base_filt_max]})
        xcorr_df = pd.concat((xcorr_df, df_add))

In [50]:
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import PairSummary
import numpy as np
import matplotlib.pyplot as plt

stims = [stimuli[2], stimuli[1], stimuli[4], stimuli[0]]

df_filt = xcorr_df.sort_values(by="filt_max")[::-1][50:100]

for ctx_idx, lgn_idx in zip(np.array(df_filt["ctx"]), np.array(df_filt["lgn"])):

    ctx_unit = pop0_sort[int(ctx_idx)]
    ctx_kernels = [SparseNoiseKernel(ctx_unit, stims[0]), 
                    SparseNoiseKernel(ctx_unit, stims[1]), 
                    StaticGratingsKernel(ctx_unit, stims[2]), 
                    StaticGratingsKernel(ctx_unit, stims[3])]

    lgn_unit = pop0_sort[int(lgn_idx)]
    lgn_kernels = [SparseNoiseKernel(lgn_unit, stims[0]), 
                    SparseNoiseKernel(lgn_unit, stims[1]), 
                    StaticGratingsKernel(lgn_unit, stims[2]), 
                    StaticGratingsKernel(lgn_unit, stims[3])]

    file_path = "G:/Shared drives/Neuropixels/Figures/RG001/PairSummary/"
    file_name = "PairSummary_{0}_{1}.png".format(str(ctx_idx), str(lgn_idx))

    PairSummary(stims, (ctx_unit, lgn_unit), (ctx_kernels, lgn_kernels), save_path=file_path+file_name)

    plt.close("all")